In [1]:
import imp
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
from sklearn import tree
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier

C:\Users\nel_1\AppData\Local\Temp\ipykernel_23052\3339299637.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
true_df = pd.read_csv('./True.csv')
fake_df = pd.read_csv('./Fake.csv')
true_df['label'] = 0
fake_df['label'] = 1
true_df = true_df[['text','label']]
fake_df = fake_df[['text','label']]
dataset = pd.concat([true_df , fake_df])
print(dataset['label'].value_counts())

1    23481
0    21417
Name: label, dtype: int64


In [3]:
dataset = dataset.sample(frac = 1)
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))
def clean_data(text):
    text = text.lower() 
    text = re.sub('[^a-zA-Z]' , ' ' , text)
    token = text.split() 
    news = [lemmatizer.lemmatize(word) for word in token if not word in stopwords]  
    clean_news = ' '.join(news) 
    
    return clean_news 

In [4]:
dataset['text'] = dataset['text'].apply(lambda x : clean_data(x))
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))
X = dataset.iloc[:5000,0]
y = dataset.iloc[:5000,1]
print(X.head())
print(y.head())


13761    ridiculous article politico pretend non biased...
12814    kiev reuters ukrainian lawmaker thursday shelv...
1740     pretty much america agree donald trump need sp...
9997     washington reuters democratic front runner hil...
15211    coincidence evidence keep magically appearing ...
Name: text, dtype: object
13761    1
12814    0
1740     1
9997     0
15211    1
Name: label, dtype: int64


In [5]:
train_X , test_X , train_y , test_y = train_test_split(X , y , test_size = 0.5 ,random_state = 0)
vec_train = vectorizer.fit_transform(train_X)
vec_train = vec_train.toarray()
vec_test = vectorizer.transform(test_X).toarray()
train_data = pd.DataFrame(vec_train , columns=vectorizer.get_feature_names_out())
test_data = pd.DataFrame(vec_test , columns= vectorizer.get_feature_names_out())
clf = RandomForestClassifier( max_depth = 5, n_estimators = 30, min_samples_split=3, max_leaf_nodes=5)
clf.fit(train_data, train_y)
predictions = clf.predict(test_data)
print(classification_report(test_y , predictions))

              precision    recall  f1-score   support

           0       0.99      0.89      0.93      1211
           1       0.90      0.99      0.94      1289

    accuracy                           0.94      2500
   macro avg       0.94      0.94      0.94      2500
weighted avg       0.94      0.94      0.94      2500

